# Dicussion 6: Arctic Communities

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

## 1. Load Data

In [2]:
fp = os.path.join('/', 'courses', 'EDS220', 'data', 'arctic_communities.geojson')
df = gpd.read_file(fp)

## 2. Check Geomtry Types

In [3]:
df.geom_type

0     MultiPolygon
1     MultiPolygon
2     MultiPolygon
3     MultiPolygon
4     MultiPolygon
5     MultiPolygon
6          Polygon
7          Polygon
8     MultiPolygon
9     MultiPolygon
10    MultiPolygon
11    MultiPolygon
12    MultiPolygon
13    MultiPolygon
14         Polygon
dtype: object

In [6]:
if (df.geom_type == 'Polygon').all():
    print("All features are polygons")
else:
    print(f"Multiple feature types: {df.geom_type.unique()}")

Multiple feature types: ['MultiPolygon' 'Polygon']


In [7]:
# Write function to check polygons
def check_polygons(df):
    if (df.geom_type == 'Polygon').all():
        print("All features are polygons")
    else:
        print(f"Multiple feature types: {df.geom_type.unique()}")

## 3. Explode polygons

In [8]:
# Explode multi-part geometries into multiple single geometries
df = df.explode(index_parts=False).reset_index()

In [9]:
# Call function to check polygons
check_polygons(df)

All features are polygons


## 4. Compute Minimum y-coordinate for Polygons

In [11]:
# a. Select first row and return type
type(df.iloc[0])

pandas.core.series.Series

In [15]:
# b. Select geometry of first row and return type
type(df.iloc[0].geometry)

shapely.geometry.polygon.Polygon

In [20]:
# c. Find minimum y-coordinate for the first polygon in df
df.iloc[0].geometry.bounds[1]

56.511035156249996

In [23]:
# d. Create function that receives 1 row of geodf and returns min-y coordinate
def min_y(row):
    row_polygon = row.geometry
    
    row_ymin = row_polygon.bounds[1]
    
    return row_ymin

In [24]:
# Use apply() to create new column of minimum y-coordinates of each polygon
df['min_y_coord'] = df.apply(func=min_y, axis=1)